In [1]:
# %% [markdown]
# # Bike Share Project — Data Acquisition, Quality Assessment, Initial Visualization
# Fill in:
# - Name, Student ID
# - City chosen
# - Historical year (2022 or 2023)
# - Portal/source link
# - Download confirmation (filenames, sizes, missing months)
#
# This notebook assumes:
# - Current station status saved by `scrape_citybikes.py` in ./data/
# - Historical trip CSVs saved locally in ./data/historical/ (one or many files)

# %%
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = Path("data")
HIST_DIR = DATA_DIR / "historical"
HIST_DIR.mkdir(parents=True, exist_ok=True)

# %% [markdown]
# ## Load current station status (from the scraper output)

# %%
# Pick the latest station_status file
station_files = sorted(DATA_DIR.glob("station_status_*.parquet")) + sorted(DATA_DIR.glob("station_status_*.csv"))
if not station_files:
    raise FileNotFoundError("No station_status files found in ./data/. Run scrape_citybikes.py first.")

latest_station_file = station_files[-1]
print("Using:", latest_station_file.name)

if latest_station_file.suffix == ".parquet":
    station_status = pd.read_parquet(latest_station_file)
else:
    station_status = pd.read_csv(latest_station_file)

station_status["timestamp"] = pd.to_datetime(station_status["timestamp"], errors="coerce", utc=True)
display(station_status.head())
station_status.info()

# %% [markdown]
# ## Task 1.2 — Load historical trip data (you download it from your city's portal)
# Put all CSVs for one full year in: `./data/historical/`
# Expected columns (names vary by city): trip_id, start_time, end_time, start_station_id, end_station_id, duration

# %%
trip_files = sorted(HIST_DIR.glob("*.csv"))
if not trip_files:
    raise FileNotFoundError("No historical trip CSVs found in ./data/historical/. Add your downloaded files first.")

print("Files found:")
for f in trip_files:
    print(" -", f.name)

# Load and concatenate
trips_raw = pd.concat((pd.read_csv(f) for f in trip_files), ignore_index=True)
display(trips_raw.head())
trips_raw.info()

# %% [markdown]
# ### Map your city's column names to standard names
# Edit the mapping below so the right columns are used.

# %%
# EDIT THIS MAPPING to match your dataset columns
COLMAP = {
    "trip_id": "trip_id",
    "start_time": "start_time",
    "end_time": "end_time",
    "start_station_id": "start_station_id",
    "end_station_id": "end_station_id",
    "duration": "duration",  # seconds or minutes depending on city
}

missing = [v for v in COLMAP.values() if v not in trips_raw.columns]
if missing:
    raise KeyError(f"Update COLMAP. Missing columns in trips_raw: {missing}")

trips = trips_raw[list(COLMAP.values())].rename(columns={v: k for k, v in COLMAP.items()}).copy()

# Parse timestamps
trips["start_time"] = pd.to_datetime(trips["start_time"], errors="coerce", utc=True)
trips["end_time"] = pd.to_datetime(trips["end_time"], errors="coerce", utc=True)

# Ensure duration numeric
trips["duration"] = pd.to_numeric(trips["duration"], errors="coerce")

display(trips.head())
trips.info()

# %% [markdown]
# ## Task 2.1 — Missing value analysis

# %%
missing_count = trips.isna().sum()
missing_pct = (missing_count / len(trips) * 100).round(3)

missing_summary = pd.DataFrame({
    "column": missing_count.index,
    "missing_count": missing_count.values,
    "missing_pct": missing_pct.values,
})

# Critical fields: timestamps + station IDs
critical_cols = ["start_time", "end_time", "start_station_id", "end_station_id"]
rows_missing_critical = trips[critical_cols].isna().any(axis=1).sum()

print("Rows with any missing critical field:", rows_missing_critical)

display(missing_summary.sort_values("missing_pct", ascending=False))

# Optional pattern check by month (systematic missingness)
trips["_month"] = trips["start_time"].dt.to_period("M")
pattern = (trips[critical_cols].isna().any(axis=1)
           .groupby(trips["_month"])
           .mean()
           .mul(100)
           .rename("pct_rows_missing_critical")
           .reset_index())
display(pattern)

# %% [markdown]
# ## Task 2.2 — Outlier detection
# Duration rules:
# - < 60 seconds
# - > 24 hours (86400 seconds)
# If your duration is in minutes, adjust thresholds accordingly.

# %%
# If your duration is in minutes, set DURATION_UNIT="minutes" and adjust thresholds.
DURATION_UNIT = "seconds"  # "seconds" or "minutes"

if DURATION_UNIT == "seconds":
    low_thr = 60
    high_thr = 24 * 3600
else:
    low_thr = 1
    high_thr = 24 * 60

duration = trips["duration"]

pctiles = duration.quantile([0.01, 0.05, 0.95, 0.99]).to_frame("duration").T
display(pctiles)

too_short = duration < low_thr
too_long = duration > high_thr

print("Too short count:", int(too_short.sum()))
print("Too long count:", int(too_long.sum()))

# Temporal anomalies
now_utc = pd.Timestamp.utcnow().tz_localize("UTC")
future_start = trips["start_time"] > now_utc
future_end = trips["end_time"] > now_utc

print("Future start_time count:", int(future_start.sum()))
print("Future end_time count:", int(future_end.sum()))

dup_trip_id = trips["trip_id"].duplicated(keep=False)
print("Duplicate trip_id count:", int(dup_trip_id.sum()))

# Spatial outliers (only possible if your trips include station coordinates; many trip datasets don't)
# If you have station lat/lon in the trip file, add checks here.

# %% [markdown]
# ## Cleaning (create `trips_clean`) + Task 2.3 log
# Make dataset-specific choices here (drop, impute, flag).
# Below is a common baseline: drop missing critical fields, remove impossible durations, drop duplicate trip_id.

# %%
before_rows = len(trips)

trips_clean = trips.copy()

# Drop rows missing critical fields
trips_clean = trips_clean.dropna(subset=critical_cols)

# Remove impossible durations
trips_clean = trips_clean[~too_short & ~too_long]

# Remove duplicate trip_id (keep first)
trips_clean = trips_clean.drop_duplicates(subset=["trip_id"], keep="first")

after_rows = len(trips_clean)

print("Rows before:", before_rows)
print("Rows after :", after_rows)
print("Removed    :", before_rows - after_rows, f"({(before_rows - after_rows)/before_rows*100:.2f}%)")

# Save cleaned data
OUT_TRIPS_CLEAN = DATA_DIR / "trips_clean.parquet"
trips_clean.to_parquet(OUT_TRIPS_CLEAN, index=False)
print("Saved:", OUT_TRIPS_CLEAN)

# %% [markdown]
# ### Task 2.3 — Cleaning decision log (write 250–400 words)
# Replace this text with your dataset-specific log:
#
# - Which outliers you removed and why
# - Which outliers you kept and why
# - How you handled missing values
# - Impact on dataset size

# %% [markdown]
# ## Task 3.1 — Temporal overview plots
# 1) Daily trip counts
# 2) Hourly patterns: weekday vs weekend (average trips per hour)
# 3) Monthly trend + moving average preview

# %%
tc = trips_clean.copy()
tc["date"] = tc["start_time"].dt.date
tc["hour"] = tc["start_time"].dt.hour
tc["weekday"] = tc["start_time"].dt.weekday  # 0=Mon
tc["is_weekend"] = tc["weekday"].isin([5, 6])

# 1) Daily
daily = tc.groupby("date").size().rename("trips").reset_index()
plt.figure()
plt.plot(pd.to_datetime(daily["date"]), daily["trips"])
plt.title("Daily trip counts")
plt.xlabel("Date")
plt.ylabel("Trips")
plt.tight_layout()
plt.show()

# %%
# 2) Hourly weekday vs weekend
hourly = tc.groupby(["is_weekend", "hour"]).size().rename("trips").reset_index()
hourly_avg = hourly.copy()
# average trips per hour across all days -> compute by (day,hour) then average
tc["day"] = tc["start_time"].dt.date
day_hour = tc.groupby(["is_weekend", "day", "hour"]).size().rename("trips").reset_index()
hour_profile = day_hour.groupby(["is_weekend", "hour"])["trips"].mean().reset_index()

plt.figure()
for flag, label in [(False, "Weekday"), (True, "Weekend")]:
    subset = hour_profile[hour_profile["is_weekend"] == flag]
    plt.plot(subset["hour"], subset["trips"], label=label)
plt.title("Average trips by hour: weekday vs weekend")
plt.xlabel("Hour of day")
plt.ylabel("Average trips")
plt.legend()
plt.tight_layout()
plt.show()

# %%
# 3) Monthly + moving average (use daily series for MA)
daily_ts = daily.copy()
daily_ts["date"] = pd.to_datetime(daily_ts["date"])
daily_ts = daily_ts.sort_values("date")
daily_ts["ma_7"] = daily_ts["trips"].rolling(7).mean()

monthly = tc.groupby(tc["start_time"].dt.to_period("M")).size().rename("trips").reset_index()
monthly["start_time"] = monthly["start_time"].dt.to_timestamp()

plt.figure()
plt.plot(daily_ts["date"], daily_ts["trips"], label="Daily")
plt.plot(daily_ts["date"], daily_ts["ma_7"], label="7-day MA")
plt.title("Daily trips with moving average")
plt.xlabel("Date")
plt.ylabel("Trips")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(monthly["start_time"], monthly["trips"])
plt.title("Monthly trip counts")
plt.xlabel("Month")
plt.ylabel("Trips")
plt.tight_layout()
plt.show()

# %% [markdown]
# ## Task 3.2 — Spatial overview
# Requires station coordinates + station usage (trips originated).
# We compute origins per start_station_id and join to station_status lat/lon.

# %%
# Aggregate usage by start station
origin_counts = tc.groupby("start_station_id").size().rename("origin_trips").reset_index()

# Join with current station metadata (id formats may differ across datasets)
stations_map = station_status.rename(columns={"station_id": "start_station_id"})
spatial = origin_counts.merge(stations_map[["start_station_id", "name", "latitude", "longitude"]], on="start_station_id", how="left")

# Usage quartiles (only for non-null counts)
spatial["usage_quartile"] = pd.qcut(spatial["origin_trips"].rank(method="first"), 4, labels=["low", "mid-low", "mid-high", "high"])

display(spatial.head())
print("Stations missing coordinates after join:", spatial["latitude"].isna().sum())

# %%
# Map with folium (interactive)
import folium

# Center map on median coordinates (drop NA)
center_lat = spatial["latitude"].dropna().median()
center_lon = spatial["longitude"].dropna().median()

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# scale marker radius
max_trips = spatial["origin_trips"].max()
def radius(x):
    if pd.isna(x) or max_trips == 0:
        return 3
    return 3 + 10 * (x / max_trips)

for _, r in spatial.dropna(subset=["latitude", "longitude"]).iterrows():
    folium.CircleMarker(
        location=[r["latitude"], r["longitude"]],
        radius=radius(r["origin_trips"]),
        popup=f"{r.get('name','')} | trips={int(r['origin_trips'])} | q={r['usage_quartile']}",
        fill=True,
    ).add_to(m)

m

# %% [markdown]
# ## Task 3.3 — Pattern discovery (choose ONE insight plot)
# Example: compare top 10 busiest vs 10 quietest origin stations.

# %%
top10 = spatial.sort_values("origin_trips", ascending=False).head(10)
bottom10 = spatial.sort_values("origin_trips", ascending=True).head(10)

compare = pd.concat([
    top10.assign(group="Top 10"),
    bottom10.assign(group="Bottom 10"),
])

# Simple bar plot
plt.figure()
# keep readable order
compare = compare.sort_values(["group", "origin_trips"], ascending=[True, False])
plt.barh(compare["group"] + " - " + compare["start_station_id"].astype(str), compare["origin_trips"])
plt.title("Top 10 vs Bottom 10 stations by trip origins")
plt.xlabel("Trips originated (count)")
plt.ylabel("Station")
plt.tight_layout()
plt.show()

# %% [markdown]
# **Caption (required, 2–3 sentences):**
# Replace this caption with your interpretation:
# - What does the plot show?
# - Why is it interesting / what hypothesis does it suggest?

# %% [markdown]
# ## Preliminary hypotheses (200–300 words)
# - Temporal hypothesis
# - Spatial hypothesis
# - Surprise


ModuleNotFoundError: No module named 'pandas'